In [ ]:
%autosave 600

In [1]:
import numpy as np
import pandas as pd

from scipy.stats import rankdata
from sklearn.metrics import roc_auc_score

import os 
from glob import glob

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

classes = train.columns[-6:]
print(classes)


Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')


In [2]:
from sklearn.preprocessing import normalize

In [9]:

files = glob(os.path.join(os.pardir, 'ensembles', 'pred*.csv'))
files = sorted(files)
files

['../ensembles/prediction_Capsule.csv',
 '../ensembles/prediction_DPCNN.csv',
 '../ensembles/prediction_LSTM.csv',
 '../ensembles/prediction_lgbm.csv',
 '../ensembles/preds_bilstmft.csv',
 '../ensembles/preds_log_reg_sgd.csv',
 '../ensembles/preds_logreg_liblinear.csv',
 '../ensembles/preds_logreg_sanket.csv',
 '../ensembles/preds_lsvc_ftrl.csv',
 '../ensembles/preds_lsvc_liblinear.csv',
 '../ensembles/preds_nb_svm.csv']

In [4]:
from sklearn.preprocessing import minmax_scale
preds_list = []
for i, f in enumerate(files):
    vals = pd.read_csv(f, usecols = classes).values
    vals = minmax_scale(vals)
    preds_list.append(vals)

In [11]:

def get_preds(coeffs):
    preds = np.zeros_like(preds_list[0])
    for c,p in zip(coeffs, preds_list):
        for i in range(len(classes)):
            preds[:, i] = np.add(preds[:, i], c*(p[:, i]))
    preds /= len(preds_list)
    labels = train[classes].values
    j = 1* roc_auc_score(labels, preds)
    # print(coeffs, j, '\n')
    return j
                   
    
carray = np.random.randint(0, 5, (500, len(files)))
ncarray = normalize(carray, norm='l1') 
h = []

for i in range(carray.shape[0]):
    q = get_preds(carray[i])
    h.append(q)


In [12]:
max = np.array(h).argmax()
print(ncarray[max], h[max])

[0.         0.13333333 0.06666667 0.         0.13333333 0.06666667
 0.26666667 0.13333333 0.         0.         0.2       ] 0.9885175847191916


In [10]:
files = glob(os.path.join(os.pardir, 'ensembles', 'test*.csv'))
files=sorted(files)
files

['../ensembles/test_average_Capsule.csv',
 '../ensembles/test_average_DPCNN.csv',
 '../ensembles/test_average_LSTM.csv',
 '../ensembles/test_average_lbgm.csv',
 '../ensembles/test_bilstmft.csv',
 '../ensembles/test_log_reg_sgd.csv',
 '../ensembles/test_logreg_liblinear.csv',
 '../ensembles/test_logreg_sanket.csv',
 '../ensembles/test_lsvc_ftrl.csv',
 '../ensembles/test_lsvc_liblinear.csv',
 '../ensembles/test_nb_svm.csv']

In [ ]:
from sklearn.preprocessing import minmax_scale
preds_list = []
for i, f in enumerate(files):
    vals = pd.read_csv(f, usecols = classes).values
    vals = minmax_scale(vals)
    preds_list.append(vals)

In [ ]:
coeffs = ncarray[max]

preds = np.zeros_like(preds_list[0])
for c,p in zip(coeffs, preds_list):
    for i in range(len(classes)):
        preds[:, i] = np.add(preds[:, i], c*(p[:, i]))
preds /= len(preds_list)


In [ ]:
submission = pd.read_csv("../input/sample_submission.csv")
# submission[classes] = minmax_scale(preds)
submission[classes] = preds
submission.to_csv("../subs/ens_10models_minmax4.csv", index = False)